In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "clientid",
"fs.azure.account.oauth2.client.secret": 'secreid',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tokenid/oauth2/token"}

dbutils.fs.mount(
source = "abfss://tokyo-olympic-data@tokyoolympicdataomkar.dfs.core.windows.net/", #container@storageacc
mount_point = "/mnt/tokyoolympic",
extra_configs = configs)

True

In [0]:
%fs
ls "/mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1752039295000
dbfs:/mnt/tokyoolympic/transform-data/,transform-data/,0,1752039313000


In [0]:
spark

In [0]:
athelets=spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/tokyoolympic/raw-data/athletes.csv")
coaches=spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/tokyoolympic/raw-data/Coaches.csv")
entriesGender=spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/tokyoolympic/raw-data/EntriesGender.csv")
teams=spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/tokyoolympic/raw-data/Teams.csv")
medals=spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/tokyoolympic/raw-data/medals.csv")

In [0]:
athelets.show()
coaches.show()
entriesGender.show()
teams.show()
medals.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athelets.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesGender.printSchema()


root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
#find the top countries with the highest no of gold medals
medals.groupBy("TeamCountry").sum("Gold").orderBy("sum(Gold)",ascending=False).show()
#find the top countries with the highest no of medals
medals.groupBy("TeamCountry").sum("Total").orderBy("sum(Total)",ascending=False).show()


+--------------------+---------+
|         TeamCountry|sum(Gold)|
+--------------------+---------+
|United States of ...|       39|
|People's Republic...|       38|
|               Japan|       27|
|       Great Britain|       22|
|                 ROC|       20|
|           Australia|       17|
|               Italy|       10|
|             Germany|       10|
|              France|       10|
|         Netherlands|       10|
|         New Zealand|        7|
|              Canada|        7|
|              Brazil|        7|
|                Cuba|        7|
|   Republic of Korea|        6|
|             Hungary|        6|
|      Czech Republic|        4|
|             Jamaica|        4|
|              Norway|        4|
|              Poland|        4|
+--------------------+---------+
only showing top 20 rows

+--------------------+----------+
|         TeamCountry|sum(Total)|
+--------------------+----------+
|United States of ...|       113|
|People's Republic...|        88|
|           

In [0]:
#Calculate the avg no of entries by gender for each discipline desc order divide by 100

entriesGender.groupBy("Discipline").avg("Female","Male").orderBy("avg(Female)","avg(Male)",ascending=True).show()

+--------------------+-----------+---------+
|          Discipline|avg(Female)|avg(Male)|
+--------------------+-----------+---------+
|Cycling BMX Frees...|       10.0|      9.0|
|Trampoline Gymnas...|       16.0|     16.0|
|      Sport Climbing|       20.0|     20.0|
|             Surfing|       20.0|     20.0|
|  Cycling BMX Racing|       24.0|     24.0|
|   Marathon Swimming|       25.0|     25.0|
|      3x3 Basketball|       32.0|     32.0|
|   Modern Pentathlon|       36.0|     36.0|
|Cycling Mountain ...|       38.0|     38.0|
|       Skateboarding|       40.0|     40.0|
|              Karate|       40.0|     42.0|
|        Canoe Slalom|       41.0|     41.0|
|    Beach Volleyball|       48.0|     48.0|
|           Triathlon|       55.0|     55.0|
|                Golf|       60.0|     60.0|
|             Archery|       64.0|     64.0|
|           Taekwondo|       65.0|     65.0|
|        Cycling Road|       70.0|    131.0|
|              Diving|       72.0|     71.0|
|         

In [0]:
athelets.repartition(1).write.mode("overwrite").format("csv").save("dbfs:/mnt/tokyoolympic/transform-data/athletes")
medals.write.mode("overwrite").format("csv").save("dbfs:/mnt/tokyoolympic/transform-data/medals")
coaches.write.mode("overwrite").format("csv").save("dbfs:/mnt/tokyoolympic/transform-data/coaches")
teams.write.mode("overwrite").format("csv").save("dbfs:/mnt/tokyoolympic/transform-data/teams")
entriesGender.write.mode("overwrite").format("csv").save("dbfs:/mnt/tokyoolympic/transform-data/entriesGender")
